In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df불러오기

In [2]:
train = pd.read_csv("/home/kunsu/est_wassup_01/main/data/origin/train.csv")
#y = pd.read_csv("/home/kunsu/est_wassup_01/main/data/features/train_E_target.csv")
area_usage = pd.read_csv("/home/kunsu/est_wassup_01/main/data/origin/gu_area_usage.csv")
car_var = pd.read_csv("/home/kunsu/est_wassup_01/main/data/origin/cars_smaller_group.csv")
bus_load_cnt = pd.read_csv("/home/kunsu/est_wassup_01/main/data/origin/bus_station_bus_count.csv")
#bus_rider = pd.read_csv("/home/kunsu/est_wassup_01/main/data/origin/bus_riders_datetime.csv")
sec_light = pd.read_csv("/home/kunsu/est_wassup_01/main/data/origin/light.csv", encoding='cp949')
cctv = pd.read_csv("/home/kunsu/est_wassup_01/main/data/origin/대구 CCTV 정보.csv", encoding='cp949')


/tmp/ipykernel_438749/1660385853.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sec_light = pd.read_csv("/home/kunsu/est_wassup_01/main/data/origin/light.csv", encoding='cp949')


In [3]:
X_df = train.copy()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
X_df.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3


시군구 확인 및 분리

In [5]:
if '시군구' in X_df.columns:
    X_df[['군', '읍면동']] = X_df['시군구'].str.split(' ', expand=True)[[1,2]]
    X_df.drop(columns=['시군구'], inplace=True)

In [6]:
# print(X_df['군'].unique())
# print(X_df['군'].nunique())  

In [7]:
# print(X_df['구(동)'].unique())
# print(X_df['구(동)'].nunique())

In [8]:
# X_df

지역 사용 확인및 머지

In [9]:
area = area_usage.copy()
area.head()

,군,공업지역,관리지역,녹지지역,농림지역,상업지역,자연환경보전지역,주거지역
0,대구(m2),41212821,0,618303816,35892842,18456376,48295620,121536744
1,중구(m2),0,0,557975,0,3121126,0,3377140
2,동구(m2),979024,0,121334591,0,3695053,35295620,20845992
3,서구(m2),4084803,0,4315272,0,1294895,0,7624438
4,남구(m2),0,0,8280222,0,1045866,0,8105365


In [10]:
area['군'] = area['군'].str.replace(r'\(.*\)', '', regex=True)
area

,군,공업지역,관리지역,녹지지역,농림지역,상업지역,자연환경보전지역,주거지역
0,대구,41212821,0,618303816,35892842,18456376,48295620,121536744
1,중구,0,0,557975,0,3121126,0,3377140
2,동구,979024,0,121334591,0,3695053,35295620,20845992
3,서구,4084803,0,4315272,0,1294895,0,7624438
4,남구,0,0,8280222,0,1045866,0,8105365
5,북구,5012475,0,65286872,0,3513338,0,20174917
6,수성구,0,0,55769762,0,1992468,0,18773294
7,달서구,11402022,0,27621767,0,2442133,0,20886684
8,달성군,19734497,0,335137355,35892842,1351497,13000000,21748914


In [11]:
# X_df[X_df['군'] == '중구']
# area[area['지역'] == '중구']

In [12]:
# print(X_df['군'].nunique())
# print(X_df['군'].unique())

In [13]:
# print(area['군'].nunique())
# print(area['군'].unique())

In [14]:
X_df = pd.merge(X_df, area, how = 'inner', on=['군'])

In [15]:
X_df

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,군,읍면동,공업지역,관리지역,녹지지역,농림지역,상업지역,자연환경보전지역,주거지역
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,중구,대신동,0,0,557975,0,3121126,0,3377140
1,ACCIDENT_00018,2019-01-01 22,화요일,맑음,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,승용,남,33세,상해없음,승용,남,69세,중상,0,1,0,0,5,중구,남산동,0,0,557975,0,3121126,0,3377140
2,ACCIDENT_00034,2019-01-02 14,수요일,맑음,단일로 - 기타,건조,차대차,측면충돌,차로위반,승용,남,85세,상해없음,승용,남,36세,경상,0,0,2,0,6,중구,동산동,0,0,557975,0,3121126,0,3377140
3,ACCIDENT_00056,2019-01-03 03,목요일,맑음,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,승용,남,68세,상해없음,승용,남,35세,경상,0,0,1,1,4,중구,남성로,0,0,557975,0,3121126,0,3377140
4,ACCIDENT_00076,2019-01-03 18,목요일,맑음,단일로 - 기타,건조,차대사람,기타,보행자보호의무위반,이륜,남,22세,상해없음,보행자,남,23세,경상,0,1,1,0,8,중구,덕산동,0,0,557975,0,3121126,0,3377140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39496,2021-12-28 17,화요일,맑음,단일로 - 기타,건조,차대차,측면충돌,안전운전불이행,승용,남,71세,상해없음,승용,남,40세,경상,0,0,1,0,3,남구,대명동,0,0,8280222,0,1045866,0,8105365
39605,ACCIDENT_39522,2021-12-29 17,수요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,50세,상해없음,이륜,남,32세,경상,0,0,1,0,3,남구,대명동,0,0,8280222,0,1045866,0,8105365
39606,ACCIDENT_39528,2021-12-29 20,수요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,여,43세,상해없음,승용,남,33세,경상,0,0,3,0,9,남구,봉덕동,0,0,8280222,0,1045866,0,8105365
39607,ACCIDENT_39537,2021-12-30 11,목요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,34세,상해없음,보행자,여,61세,경상,0,0,1,0,3,남구,대명동,0,0,8280222,0,1045866,0,8105365


차량용도 확인 및 머지

In [16]:
car = car_var.copy()
car.head()

,군,읍면동,승용,승합,화물,특수,소계
0,남구,대명동,29753,1244,4953,118,36068
1,남구,봉덕동,14627,418,1792,48,16885
2,남구,이천동,4207,116,504,14,4841
3,달서구,갈산동,1201,138,807,4,2150
4,달서구,감삼동,12204,289,1892,40,14425


In [17]:
# print(X_df['읍면동'].nunique())
# print(X_df['읍면동'].unique())

In [18]:
# print(car['읍면동'].nunique())
# print(car['읍면동'].unique())

In [19]:
# unique_in_df1 = X_df[~X_df['읍면동'].isin(car['읍면동'])]['읍면동']

# # 결과 출력
# print("X_df에는 있고 car에는 없는것")
# print(unique_in_df1.unique())
# print(unique_in_df1.nunique())

In [20]:
# unique_in_df1 = car[~car['읍면동'].isin(X_df['읍면동'])]['읍면동']

# # 결과 출력
# print("car에는 있고 X_df에는 없는것")
# print(unique_in_df1.unique())
# print(unique_in_df1.nunique())

In [21]:
X_df = pd.merge(X_df, car, how = 'inner', on=['군', '읍면동'])
X_df.drop('소계', axis=1, inplace=True)

In [22]:
X_df

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,군,읍면동,공업지역,관리지역,녹지지역,농림지역,상업지역,자연환경보전지역,주거지역,승용,승합,화물,특수
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7
1,ACCIDENT_00090,2019-01-04 04,금요일,맑음,교차로 - 교차로부근,건조,차대차,추돌,안전운전불이행,승용,남,19세,상해없음,승용,남,71세,경상,0,0,1,0,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7
2,ACCIDENT_00296,2019-01-10 15,목요일,맑음,교차로 - 교차로부근,건조,차대사람,횡단중,보행자보호의무위반,화물,남,70세,상해없음,보행자,여,73세,중상,0,1,0,0,5,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7
3,ACCIDENT_00459,2019-01-15 17,화요일,맑음,단일로 - 터널,건조,차대사람,횡단중,보행자보호의무위반,승용,남,37세,상해없음,보행자,여,46세,경상,0,0,1,0,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7
4,ACCIDENT_00588,2019-01-19 09,토요일,맑음,단일로 - 기타,건조,차대사람,기타,안전운전불이행,화물,여,64세,상해없음,보행자,여,79세,경상,0,0,1,0,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39313,2021-12-22 20,수요일,맑음,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,승용,남,62세,상해없음,승용,남,30세,경상,0,0,1,0,3,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48
39605,ACCIDENT_39323,2021-12-23 06,목요일,맑음,교차로 - 교차로안,건조,차대차,정면충돌,안전운전불이행,승합,남,50세,상해없음,승용,여,64세,경상,0,0,1,0,3,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48
39606,ACCIDENT_39435,2021-12-26 22,일요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,45세,상해없음,승용,여,43세,경상,0,0,2,0,6,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48
39607,ACCIDENT_39490,2021-12-28 15,화요일,맑음,교차로 - 교차로부근,건조,차대차,측면충돌,신호위반,승용,남,54세,부상신고,승용,여,43세,경상,0,0,1,1,4,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48


# 버스경로 수 추가
동 안맞아서 줄어들음, 혹은 nan값 생성
동은 합쳐서 군으로만 머지해봄

In [23]:
bus = bus_load_cnt.copy()

In [24]:
bus.rename(columns={'구/군': '군', '동': '읍면동'}, inplace=True)

In [25]:
bus

,index,군,읍면동,경유노선수
0,0,남구,대명10동,61
1,1,남구,대명11동,40
2,2,남구,대명1동,18
3,3,남구,대명2동,33
4,4,남구,대명3동,69
...,...,...,...,...
138,138,중구,동인3가동,6
139,139,중구,삼덕동,37
140,140,중구,성내1동,110
141,141,중구,성내2동,127


In [26]:
 print(X_df['읍면동'].nunique())
 print(bus['읍면동'].nunique())

196
143


In [27]:
# print(X_df['읍면동'].unique())

In [28]:
# bus['읍면동'].unique()

In [29]:
# df1에는 있지만 df2에는 없는 읍면동의 값을 찾기
# unique_in_df1 = X_bus[~X_bus['읍면동'].isin(bus_bus['읍면동'])]['읍면동']
unique_in_df1 = X_df[~X_df['읍면동'].isin(bus['읍면동'])]['읍면동']

# 결과 출력
# print(unique_in_df1.unique())
# print(unique_in_df1.nunique())

In [30]:
unique_in_df2 = bus[~bus['읍면동'].isin(X_df['읍면동'])]['읍면동']

# 결과 출력
# print(unique_in_df2.unique())
# print(unique_in_df2.nunique())

In [31]:
b1 = bus.groupby('군').sum().drop(['읍면동'], axis=1)
b1

,index,경유노선수
군,,
남구,78,592
달서구,588,1930
달성군,369,1538
동구,1110,2006
북구,1860,2193
서구,1666,743
수성구,2714,1918
중구,1768,607


여기서 머지를 하면서 row가 줄어들음
inner - 줄음
outer - 늘음



In [32]:
X_df = pd.merge(X_df, b1, how = 'left', on=['군'])
X_df.drop('index', axis=1, inplace=True)

In [33]:
X_df

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,군,읍면동,공업지역,관리지역,녹지지역,농림지역,상업지역,자연환경보전지역,주거지역,승용,승합,화물,특수,경유노선수
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
1,ACCIDENT_00090,2019-01-04 04,금요일,맑음,교차로 - 교차로부근,건조,차대차,추돌,안전운전불이행,승용,남,19세,상해없음,승용,남,71세,경상,0,0,1,0,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
2,ACCIDENT_00296,2019-01-10 15,목요일,맑음,교차로 - 교차로부근,건조,차대사람,횡단중,보행자보호의무위반,화물,남,70세,상해없음,보행자,여,73세,중상,0,1,0,0,5,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
3,ACCIDENT_00459,2019-01-15 17,화요일,맑음,단일로 - 터널,건조,차대사람,횡단중,보행자보호의무위반,승용,남,37세,상해없음,보행자,여,46세,경상,0,0,1,0,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
4,ACCIDENT_00588,2019-01-19 09,토요일,맑음,단일로 - 기타,건조,차대사람,기타,안전운전불이행,화물,여,64세,상해없음,보행자,여,79세,경상,0,0,1,0,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39313,2021-12-22 20,수요일,맑음,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,승용,남,62세,상해없음,승용,남,30세,경상,0,0,1,0,3,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48,592
39605,ACCIDENT_39323,2021-12-23 06,목요일,맑음,교차로 - 교차로안,건조,차대차,정면충돌,안전운전불이행,승합,남,50세,상해없음,승용,여,64세,경상,0,0,1,0,3,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48,592
39606,ACCIDENT_39435,2021-12-26 22,일요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,45세,상해없음,승용,여,43세,경상,0,0,2,0,6,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48,592
39607,ACCIDENT_39490,2021-12-28 15,화요일,맑음,교차로 - 교차로부근,건조,차대차,측면충돌,신호위반,승용,남,54세,부상신고,승용,여,43세,경상,0,0,1,1,4,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48,592


보안등 확인

In [34]:
sec_light.head()

,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치연도,설치형태
0,대명1동1,1,대구광역시 남구 현충로 155,대구광역시 남구 대명동 1722-4,35.846703,128.579803,2016.0,한전주
1,대명1동2,1,대구광역시 남구 현충로31길 10-1,대구광역시 남구 대명동 1723-11,35.846863,128.579406,2016.0,한전주
2,대명1동3,1,대구광역시 남구 현충로31길 9-8,대구광역시 남구 대명동 1722-56,35.846341,128.579342,2017.0,건축물
3,대명1동4,2,대구광역시 남구 현충로31길 19-11,대구광역시 남구 대명동 1722-41,35.846368,128.578922,2016.0,한전주
4,대명1동5,1,대구광역시 남구 현충로29길 22-2,대구광역시 남구 대명동 1721-22,35.845995,128.578858,2016.0,한전주


In [35]:
# sec = sec_light['소재지지번주소'].str.split(' ',expand=True).str[:1]
# #sec_light[['시', '군', '읍면동', '지번']] = sec
# #sec_light
# sec

In [36]:
# sec_light['시'] = sec_light['소재지지번주소'].str.split(' ').str[0:1].astype(int)
# sec_light['군'] = sec_light['소재지지번주소'].str.split(' ').str[1:2].astype(int)
# sec_light['읍면동'] = sec_light['소재지지번주소'].str.split(' ').str[2:3].astype(int)
# sec_light['지번'] = sec_light['소재지지번주소'].str.split(' ').str[-1].astype(int)

In [37]:
cctv

,무인교통단속카메라관리번호,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,설치장소,단속구분,제한속도,단속구간위치구분,과속단속구간길이,보호구역구분,설치연도
0,주차12,대구광역시,남구,시도,NaN,대명로,3,대구광역시 남구 대명로 162,대구광역시 남구 대명동 903-13,35.838857,128.575418,안지랑네거리1 버스정류장,4,0,NaN,NaN,99.0,2010.0
1,주차13,대구광역시,남구,시도,NaN,봉덕로,3,대구광역시 남구 봉덕로 135,대구광역시 남구 봉덕동 853-8,35.844653,128.604619,효성백년가약,4,0,NaN,NaN,99.0,2013.0
2,주차14,대구광역시,남구,시도,NaN,대봉로,3,대구광역시 남구 봉덕로 134,대구광역시 남구 봉덕동 866-46,35.844183,128.604815,강변코오롱하늘채,4,0,NaN,NaN,99.0,2013.0
3,주차15,대구광역시,남구,시도,NaN,큰골길,3,대구광역시 남구 큰골길 200,대구광역시 남구 대명동 2680-17,35.834798,128.585284,보성청록타운,4,0,NaN,NaN,99.0,2014.0
4,주차16,대구광역시,남구,시도,NaN,큰골길,3,대구광역시 남구 큰골길 243,대구광역시 남구 봉덕동 1616,35.832728,128.588324,환경차고지,4,0,NaN,NaN,99.0,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,H2418,대구광역시,북구,특별시도,NaN,태암남로,1,대구광역시 북구 태암남로 41,대구광역시 북구 구암동 671-8,35.925294,128.552023,구암고교교차로(면허시험장삼거리→구암역),2,40,NaN,NaN,99.0,2021.0
1061,H2419,대구광역시,북구,특별시도,NaN,내곡로,1,대구광역시 북구 내곡로 90,대구광역시 북구 사수동 812,35.900977,128.513825,사수초교앞(삼영초교→사수중학교),2,30,NaN,NaN,2.0,2021.0
1062,H4145,대구광역시,북구,특별시도,NaN,대현로,2,대구광역시 북구 대현로 100,대구광역시 북구 대현동 270-8,35.882191,128.609791,신암초교 남측(공고R)(경대교→공고네거리),2,30,NaN,NaN,2.0,2021.0
1063,H4146,대구광역시,북구,특별시도,NaN,대현로,1,대구광역시 북구 대현로 118,대구광역시 북구 대현동 270-3,35.881737,128.611651,신암초교 남측(경대교)(공고네거리→경대교),2,30,NaN,NaN,2.0,2021.0


In [38]:
cctv['시군구명'].unique()

array(['남구', '달성', '달서구', '중구', '달성군', '북구', '동구', '수성구', '서구', '군위군'],
      dtype=object)

In [39]:
cctv.drop(['무인교통단속카메라관리번호', '시도명', '도로종류', '도로노선번호', '도로노선명', '소재지도로명주소', '소재지지번주소', '위도', '경도', '설치장소'], axis=1, inplace=True)

In [40]:
c1 = cctv.groupby('시군구명').mean()
c1

,도로노선방향,단속구분,제한속도,단속구간위치구분,과속단속구간길이,보호구역구분,설치연도
시군구명,,,,,,,
군위군,1.416667,9.416667,60.416667,1.500000,11.090,78.791667,2019.583333
남구,2.297872,3.053191,18.936170,NaN,NaN,73.180851,2017.414894
달서구,2.180505,4.949458,23.862816,1.500000,NaN,37.305825,2017.267148
달성,1.454545,8.754545,48.909091,1.500000,6494.800,2.000000,2018.400000
달성군,1.460177,12.000000,49.823009,1.500000,8.663,60.371681,2018.238938
동구,1.489130,16.543478,55.434783,1.571429,6.697,70.521739,2017.869565
북구,1.440000,3.256000,45.920000,2.000000,8.857,69.512000,2018.720000
서구,1.527273,1.909091,47.272727,NaN,NaN,76.072727,2018.290909
수성구,1.490196,8.421569,50.784314,1.571429,NaN,76.176471,2018.333333


# 합친거 전처리 어떻게 할까

In [41]:
# import sys
# sys.path.insert(0, '.')

# from datasets.utils import custom_X_preprocess_cat
# from datasets import HomeData

In [42]:
# from datasets.homedata import HomeData
# from configs.preprocess_config import config

In [43]:
# home = HomeData(**config.get('input_data'),**config.get('options'))

In [44]:
columns_to_drop = ['사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 상해정도','피해운전자 차종','피해운전자 성별','피해운전자 연령','피해운전자 상해정도','사망자수','중상자수','경상자수','부상자수']
X_df.drop(columns=columns_to_drop, inplace=True)

In [45]:
X_df

,ID,사고일시,요일,기상상태,도로형태,노면상태,사고유형,가해운전자 연령,ECLO,군,읍면동,공업지역,관리지역,녹지지역,농림지역,상업지역,자연환경보전지역,주거지역,승용,승합,화물,특수,경유노선수
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,단일로 - 기타,건조,차대사람,51세,5,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
1,ACCIDENT_00090,2019-01-04 04,금요일,맑음,교차로 - 교차로부근,건조,차대차,19세,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
2,ACCIDENT_00296,2019-01-10 15,목요일,맑음,교차로 - 교차로부근,건조,차대사람,70세,5,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
3,ACCIDENT_00459,2019-01-15 17,화요일,맑음,단일로 - 터널,건조,차대사람,37세,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
4,ACCIDENT_00588,2019-01-19 09,토요일,맑음,단일로 - 기타,건조,차대사람,64세,3,중구,대신동,0,0,557975,0,3121126,0,3377140,3004,86,626,7,607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39313,2021-12-22 20,수요일,맑음,단일로 - 기타,건조,차대차,62세,3,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48,592
39605,ACCIDENT_39323,2021-12-23 06,목요일,맑음,교차로 - 교차로안,건조,차대차,50세,3,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48,592
39606,ACCIDENT_39435,2021-12-26 22,일요일,맑음,단일로 - 기타,건조,차대차,45세,6,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48,592
39607,ACCIDENT_39490,2021-12-28 15,화요일,맑음,교차로 - 교차로부근,건조,차대차,54세,4,남구,봉덕동,0,0,8280222,0,1045866,0,8105365,14627,418,1792,48,592


In [46]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer

x_scaler1 = MinMaxScaler()#MinMaxScaler()
y_scaler1 = MinMaxScaler()#None#MinMaxScaler()

In [47]:
from typing import Type
from sklearn.preprocessing import OneHotEncoder
from typing import Literal
from sklearn.base import BaseEstimator

def custom_X_preprocess_cat(X_df: Type[pd.DataFrame]) -> Type[pd.DataFrame]:
    X_df = X_df.copy()
    
    # ['사고일시', '사고유형', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']
    # 사고일시 yyyy-mm-dd hh datetime - 시간 컬럼 생성
    if '사고일시' in X_df.columns:
        X_df['시간'] = X_df['사고일시'].str.split(' ').str[-1].astype(int)
        X_df.drop(columns=['사고일시'], inplace=True)
    
    # 도로형태 e.g. 교차로 - 교차로안 -> 도로형태와 세부도로형태로 분리
    if '도로형태' in X_df.columns:
        X_df[['도로형태-1', '도로형태-2']] = X_df['도로형태'].str.split(' - ', expand=True)
        X_df.drop(columns=['도로형태'], inplace=True)
    
    # 시군구 대전데이터 한정으로, 군, 동 정보만 남긴다.
    if '시군구' in X_df.columns:
        # 군, 구(동)
        X_df[['군', '구(동)']] = X_df['시군구'].str.split(' ', expand=True)[[1,2]]
        X_df.drop(columns=['시군구'], inplace=True)
        
        # df_test 의 unique values 와 df_train 의 unique values 가 다르므로 우선 제외시킴
        X_df.drop(columns=['구(동)'], inplace=True)
    
    if '노면상태' in X_df.columns:
        # 건조 젖음/습기 기타 서리/결빙 침수 적설
        X_df['노면상태'] = X_df['노면상태'].map(lambda x: 
        0 if x == '건조' else
        1 if x == '기타' else
        2 if x == '젖음/습기' else
        3 if x == '서리/결빙' else
        4
        )
    
    if '기상상태' in X_df.columns:
        # 맑음 비 흐림 기타 눈
        X_df['기상상태'] = X_df['기상상태'].map(lambda x: 
        0 if x == '맑음' else
        1 if x == '기타' else
        2 if x == '흐림' else
        3 if x == '비' else
        4
        )
    
    if '요일' in X_df.columns:
        # 월 화 수 목 금 토 일
        X_df['요일'] = X_df['요일'].map(lambda x: 
        1 if x == '토요일' or  x == '일요일' else
        0 
        )

    if '가해운전자 연령' in X_df.columns:
        X_df['가해운전자 연령'] = X_df['가해운전자 연령'].str.extract('(\d+)', expand=False).astype(float)
        X_df['가해운전자 연령'] = X_df['가해운전자 연령'].map(lambda x: 
        0 if x < 10 else
        1 if x >= 10 and x < 20  else
        2 if x >= 20 and x < 30  else
        3 if x >= 30 and x < 40  else
        4 if x >= 40 and x < 50  else
        5 if x >= 50 and x < 60  else
        6 if x >= 60 and x < 70  else
        7 if x >= 70 and x < 80  else
        8 if x >= 80 and x < 90  else
        9 
        )
        
    if '사고유형' in X_df.columns:
        # 월 화 수 목 금 토 일
        X_df['사고유형'] = X_df['사고유형'].map(lambda x: 
        0 if x == '차대사람' else
        1 if x == '차대차' else
        2 if x == '차랑단독' else
        3
    )
    
    return X_df

class HomeData():
    def __init__(self, X_df: pd.DataFrame):
        X_df.set_index('ID', drop=True, inplace=True)
        self.target = X_df['ECLO']
        self.X_df = X_df.drop(['ECLO'], axis=1)


    fill_num_strategy: Literal['mean', 'min', 'max'] = 'min'
    x_scaler: BaseEstimator = x_scaler1
    y_scaler: BaseEstimator = y_scaler1

    
    def _scale_X(self, X_df: pd.DataFrame):
        self.x_scaler.fit(X_df)
        return self.x_scaler.transform(X_df).astype(dtype=np.float32)

    def _scale_Y(self, target_origin:iter):
        target = pd.DataFrame(target_origin)
        if self.y_scaler == None:
            return target, target
        self.y_scaler.fit(target)
        return target_origin, pd.DataFrame(self.y_scaler.transform(target).astype(dtype=np.float32))

    def _X_preprocess(self, X_df: pd.DataFrame):
        # Custom X preprocess for cat data - label encoded or cat objects
        X_df = custom_X_preprocess_cat(X_df)

        # Numeric
        df_num = X_df.select_dtypes(include=['number'])
        if self.fill_num_strategy == 'mean':
            fill_values = df_num.mean(axis=1)
        elif self.fill_num_strategy == 'min':
            fill_values = df_num.min(axis=1)
        elif self.fill_num_strategy == 'max':
            fill_values = df_num.max(axis=1)
        df_num.fillna(fill_values, inplace=True)
        if self.x_scaler is not None:
            df_num = pd.DataFrame(self._scale_X(df_num), columns=df_num.columns)

        # Categorical
        df_cat = X_df.select_dtypes(include=['object'])
        enc = OneHotEncoder(dtype=np.float32, sparse_output=False, drop='if_binary', handle_unknown='ignore')
        df_cat_onehot = pd.DataFrame(enc.fit_transform(df_cat))

        df_arr = pd.concat([df_num.reset_index(drop=True), df_cat_onehot.reset_index(drop=True)],axis=1).set_index(X_df.index)

        return df_arr



    def preprocess(self):
        """_summary_

        Returns:
            trn_X, scaled_target, target_origin, tst_X
        """
        trn_df = self.X_df.copy()
        target = self.target
        # X Features
        trn_X = self._X_preprocess(trn_df)
        # tst_X = self._X_preprocess(tst_df)
        
        # Y Feature
        target_origin, scaled_target = self._scale_Y(target)

        return trn_X, scaled_target, target_origin#, tst_X

In [48]:
home = HomeData(X_df)

In [49]:
trn_X, scaled_target, target_origin = home.preprocess()

In [50]:
scaled_target = scaled_target.set_index(trn_X.index)

In [51]:
scaled_target

,0
ID,
ACCIDENT_00000,0.406877
ACCIDENT_00090,-0.512252
ACCIDENT_00296,0.406877
ACCIDENT_00459,-0.512252
ACCIDENT_00588,-0.512252
...,...
ACCIDENT_39313,-0.512252
ACCIDENT_39323,-0.512252
ACCIDENT_39435,0.731069


In [52]:
trn_X

,요일,기상상태,노면상태,사고유형,가해운전자 연령,공업지역,관리지역,녹지지역,농림지역,상업지역,자연환경보전지역,주거지역,승용,승합,화물,특수,경유노선수,시간,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ACCIDENT_00000,-0.572754,-0.307808,-0.295908,-1.857027,0.341951,-1.507126,0.0,-2.117227,-0.274052,0.730147,-0.507334,-1.933276,-1.150233,-1.212355,-1.004286,-1.353514,-1.759514,-2.161822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
ACCIDENT_00090,-0.572754,-0.307808,-0.295908,0.284238,-2.094875,-1.507126,0.0,-2.117227,-0.274052,0.730147,-0.507334,-1.933276,-1.150233,-1.212355,-1.004286,-1.353514,-1.759514,-1.698002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACCIDENT_00296,-0.572754,-0.307808,-0.295908,-1.857027,1.391227,-1.507126,0.0,-2.117227,-0.274052,0.730147,-0.507334,-1.933276,-1.150233,-1.212355,-1.004286,-1.353514,-1.759514,0.137504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [53]:
trn_X.to_csv('./data/features/train_X.csv')

In [54]:
scaled_target.to_csv('./data/features/train_target.csv')